1. Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from tqdm import tqdm

2. Leitura do csv com os dados mockados

In [2]:
df_final_pred = pd.read_csv("df_final_pred.csv")

3. Preparação dos dados

In [ ]:
print("PASSO INICIAL: Criando uma cópia do DataFrame original...")
df_final = df_final_pred.copy()
print("Cópia do DataFrame original criada com sucesso.")
print("-" * 50)

print("ETAPA 1: Preparando os dados...")
df_final['trecho'] = df_final['place_origin_departure_mock'] + ' - ' + df_final['place_destination_departure_mock']
df_ratings = df_final.groupby(['fk_contact_mock', 'trecho']).size().reset_index(name='n_compras')
print("DataFrame de avaliações implícitas criado.")
print("-" * 50)

4. Integração com a biblioteca scikit-surprise

In [ ]:
print("ETAPA 2: Integrando com Scikit-Surprise...")
reader = Reader(rating_scale=(df_ratings['n_compras'].min(), df_ratings['n_compras'].max()))
data = Dataset.load_from_df(df_ratings[['fk_contact_mock', 'trecho', 'n_compras']], reader)
print("Dados carregados no formato do Surprise com sucesso.")
print("-" * 50)

5. Treinamento do modelo de Machine Learning (SVD)

In [ ]:
print("ETAPA 3: Treinando o modelo SVD...")
trainset = data.build_full_trainset()
algo = SVD(random_state=42)
algo.fit(trainset)
print("Modelo treinado com sucesso.")
print("-" * 50)

6. Geração de Recomendações (Otimizado com Lotes)

In [ ]:
print("ETAPA 4 (Lotes): Gerando recomendações com baixo uso de memória...")

user_factors = algo.pu.astype(np.float32) 
item_factors = algo.qi.astype(np.float32)
map_item_index_to_name = {i: trainset.to_raw_iid(i) for i in range(item_factors.shape[0])}

batch_size = 5000  
recomendacoes_por_cliente = {}
n_users = user_factors.shape[0]

print(f"Processando {n_users} usuários em lotes de {batch_size}...")
for i in tqdm(range(0, n_users, batch_size), desc="Processando lotes de clientes"):
    # Define o início e o fim do lote atual
    start_index = i
    end_index = min(i + batch_size, n_users)
    
    # Pega o subconjunto de fatores de usuário para o lote atual
    batch_user_factors = user_factors[start_index:end_index]
    
    # Calcula as pontuações APENAS para o lote atual (usa pouca memória)
    batch_scores = np.dot(batch_user_factors, item_factors.T)
    
    # Encontra o melhor item para cada usuário NO LOTE atual
    batch_best_item_indices = np.argmax(batch_scores, axis=1)
    
    # Mapeia os resultados do lote de volta para os IDs originais
    for j, user_inner_id_in_batch in enumerate(range(start_index, end_index)):
        raw_user_id = trainset.to_raw_uid(user_inner_id_in_batch)
        best_item_index = batch_best_item_indices[j]
        best_trecho_name = map_item_index_to_name[best_item_index]
        recomendacoes_por_cliente[raw_user_id] = best_trecho_name

print("\nMapeando resultados finais para o DataFrame...")
df_final['trecho_recomendado'] = df_final['fk_contact_mock'].map(recomendacoes_por_cliente)
print("Coluna 'trecho_recomendado' adicionada com sucesso!")
print("-" * 50)


7. Verificação dos resultados

In [ ]:
print("Visualizando o DataFrame final com a nova coluna:")
print(df_final[['fk_contact_mock', 'trecho', 'trecho_recomendado']].head(10))

clientes_unicos = df_final['fk_contact_mock'].unique()
if len(clientes_unicos) > 0:
    cliente_exemplo_1 = clientes_unicos[0]
    print(f"\nVerificando as recomendações para o cliente: '{cliente_exemplo_1}'")
    print(df_final[df_final['fk_contact_mock'] == cliente_exemplo_1][['fk_contact_mock', 'trecho', 'trecho_recomendado']])
else:
    print("\nNão há clientes no DataFrame para exibir como exemplo.")

PASSO INICIAL: Criando uma cópia do DataFrame original...
Cópia do DataFrame original criada com sucesso.
--------------------------------------------------
PASSO 1: Preparando os dados...
DataFrame de avaliações implícitas criado.
--------------------------------------------------
PASSO 2: Integrando com Scikit-Surprise...
Dados carregados no formato do Surprise com sucesso.
--------------------------------------------------
PASSO 3: Treinando o modelo SVD...
Modelo treinado com sucesso.
--------------------------------------------------
PASSO 4 (Lotes): Gerando recomendações com baixo uso de memória...
Processando 375590 usuários em lotes de 5000...


Processando lotes de clientes: 100%|██████████| 76/76 [00:16<00:00,  4.65it/s]



Mapeando resultados finais para o DataFrame...
Coluna 'trecho_recomendado' adicionada com sucesso!
--------------------------------------------------
Visualizando o DataFrame final com a nova coluna:
  fk_contact_mock                 trecho       trecho_recomendado
0       contact_1   cidade_1 - cidade_49   cidade_89 - cidade_878
1       contact_2  cidade_2 - cidade_101  cidade_216 - cidade_144
2       contact_3  cidade_3 - cidade_269   cidade_34 - cidade_117
3       contact_4  cidade_4 - cidade_482     cidade_1 - cidade_42
4       contact_5  cidade_5 - cidade_628     cidade_6 - cidade_55
5       contact_6   cidade_1 - cidade_49  cidade_631 - cidade_680
6       contact_7   cidade_1 - cidade_49     cidade_49 - cidade_1
7       contact_8  cidade_6 - cidade_165  cidade_335 - cidade_145
8       contact_3  cidade_3 - cidade_269   cidade_34 - cidade_117
9       contact_9  cidade_7 - cidade_137    cidade_7 - cidade_394

Verificando as recomendações para o cliente: 'contact_1'
        fk_cont

In [16]:
df_final.head()

,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,...,ano,perfil_cliente,perfil_cliente_num,mes,pred7d_proba,pred7d_pred,pred30d_proba,pred30d_pred,trecho,trecho_recomendado
0,aa34ed7fd0a6b405df2df1bf9f8d68e6df9b9a868a6181...,ceea0de820a6379f2c4215bddaec66c33994b304607e56...,2021-02-23,20:08:25,7688b6ef52555962d008fff894223582c484517cea7da4...,23765fc69c4e3c0b10f5d15471dc2245e2a19af16b513f...,0,0,48449a14a4ff7d79bb7a1b6f3d488eba397c36ef25634c...,1,...,2021,Baixo,1,2,0.239338,1.0,0.363879,1.0,cidade_1 - cidade_49,cidade_89 - cidade_878
1,948356b25b90c0c87c147cead27483c481edda1dacc4c8...,37f13b6dae3cfd6e82caf88c4e361a5b2035b66c383ba2...,2021-02-11,22:25:15,d26eae87829adde551bf4b852f9da6b8c3c2db9b65b8b6...,482d9673cfee5de391f97fde4d1c84f9f8d6f2cf0784fc...,0,0,1dfacb2ea5a03e0a915999e03b5a56196f1b1664d2f768...,1,...,2021,Baixo,1,2,0.189359,1.0,0.303669,1.0,cidade_2 - cidade_101,cidade_216 - cidade_144
2,2ee9d0978acb5e113d0b3f846ab3f88c5a426321da8f87...,e15109b0b8e9f6f1554e560837eb55543f035f91d8be4f...,2021-02-19,19:11:40,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,be47addbcb8f60566a3d7fd5a36f8195798e2848b36819...,0,0,1d0ebea552eb43d0b1e1561f6de8ae92e3de7f1abec523...,1,...,2021,Baixo,1,2,0.188411,1.0,0.287753,1.0,cidade_3 - cidade_269,cidade_34 - cidade_117
3,929cd361c225ec5d3510e14e8582fdcc61a24383cdb7a7...,f914295cdc6f40aae952cd1457650eac8a6f74a57a4923...,2021-07-02,11:41:19,90b5bc7f03c840b2efddb22ffdfc37dd12cb391b49aa0f...,f6103ca1e01bd200a9258a366b7e8c22a542e771bf11a0...,0,0,c6f3ac57944a531490cd39902d0f777715fd005efac9a3...,1,...,2021,Baixo,1,7,0.136300,0.0,0.249249,1.0,cidade_4 - cidade_482,cidade_1 - cidade_42
4,f08c3f551a19f1ce13525825dbf0d0ce9c3492da92bbb2...,17427df9b085e934834e8fe1b6bce73f46f75facfa7f91...,2022-07-14,10:16:52,20ca98162ba780883712eb701c84e4c06f73aba78e9039...,a77b6cbdf6fae1676369dea1e1ea675e4c2400c9e43bd5...,0,0,96061e92f58e4bdcdee73df36183fe3ac64747c81c26f6...,1,...,2022,Baixo,1,7,0.129553,0.0,0.236681,0.0,cidade_5 - cidade_628,cidade_6 - cidade_55


In [15]:
df_final.to_csv("df_final_trecho.csv", index=False)